In [27]:
from src.data.preprocess_dataset import create_dataloader
from src.models.model import HGNModel, Validation
from src.models.document_retrieval import DocumentRetrieval
import torch
import json
import os

In [28]:
data_path = 'data/'
model_path = 'models'

print("Preprocessing data")
hotpotqa_path = 'external/'
print("Loading HotpotQA")
#hotpot_dev_distractor_v1
with open(os.path.join(data_path, hotpotqa_path, "hotpot_dev_distractor_v1.json"), "r") as f:
    hotpot = json.load(f)
hotpot = hotpot

Preprocessing data
Loading HotpotQA


In [ ]:
device = 'cuda'
doc_retr_model_path = '/workspace/ml-workspace/thesis_git/HSGN/models/doc_retrieval'
print("Loading the document retrieval model")
doc_retr = DocumentRetrieval(device, doc_retr_model_path)
print("Computing the relevant documents")
dict_ins2dict_doc2pred = doc_retr.predict_relevant_docs(hotpot)

Loading the document retrieval model


In [ ]:
# import pickle
# with open('dict_ins2dict_doc2pred_top4.p', 'wb') as f:
#     pickle.dump(dict_ins2dict_doc2pred, f)

In [29]:
import pickle
with open('dict_recall2_last.pickle', 'rb') as f:
    dict_ins2dict_doc2pred = pickle.load(f)

In [30]:
from src.data.graph_creation import Dataset
from src.data.preprocessing import NER_stanza
from src.data.preprocessing import SRL

In [31]:
ner = NER_stanza()
srl = SRL()
list_ent_query = ner.extract_named_entities_from_query(hotpot)
print("Extracting named entities")
list_hotpot_ner = ner.extract_named_entities(hotpot, dict_ins2dict_doc2pred)
print("Extracting SRL arguments from the query")
dict_ins_query_srl_triples = srl.extract_srl_from_query(hotpot)
print("Extracting SRL arguments")
dict_ins_doc_sent_srl_triples = srl.extract_srl(hotpot, dict_ins2dict_doc2pred)
print("Data loaded. Creating graphs")

/opt/conda/envs/thesis/lib/python3.7/site-packages/stanza/utils/resources.py:53: ResourceWarning: unclosed file <_io.BufferedReader name='/root/stanza_resources/resources.json'>
  data = open(path, 'rb').read()
/opt/conda/envs/thesis/lib/python3.7/site-packages/stanza/utils/resources.py:224: ResourceWarning: unclosed file <_io.TextIOWrapper name='/root/stanza_resources/resources.json' mode='r' encoding='UTF-8'>
  resources = json.load(open(os.path.join(dir, 'resources.json')))
2020-09-10 02:02:11 INFO: Downloading default packages for language: en (English)...
/opt/conda/envs/thesis/lib/python3.7/site-packages/stanza/utils/resources.py:53: ResourceWarning: unclosed file <_io.BufferedReader name='/root/stanza_resources/en/default.zip'>
  data = open(path, 'rb').read()
2020-09-10 02:02:11 INFO: File exists: /root/stanza_resources/en/default.zip.
2020-09-10 02:02:17 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-09-10 02:02:17 INFO: Loading these models for la

Extracting named entities


  0%|          | 2/7405 [00:00<07:51, 15.72it/s]

Extracting SRL arguments from the query


  0%|          | 0/7405 [00:00<?, ?it/s]

Extracting SRL arguments


100%|██████████| 7405/7405 [54:27<00:00,  2.27it/s]  

Data loaded. Creating graphs


In [32]:
import pickle
with open('list_ent_query_giwon_last.p', 'wb+') as f:
    pickle.dump(list_ent_query, f)
with open('list_hotpot_ner_giwon_last.p', 'wb+') as f:
    pickle.dump(list_hotpot_ner, f)
with open('dict_ins_query_srl_triples_giwon_last.p', 'wb+') as f:
    pickle.dump(dict_ins_query_srl_triples, f)
with open('dict_ins_doc_sent_srl_triples_giwon_last.p', 'wb+') as f:
    pickle.dump(dict_ins_doc_sent_srl_triples, f)

In [ ]:
# import pickle
# with open('list_ent_query_v3.p', 'rb') as f:
#     list_ent_query = pickle.load(f)
# with open('list_hotpot_ner_v3.p', 'rb') as f:
#     list_hotpot_ner = pickle.load(f)
# with open('dict_ins_query_srl_triples_v3.p', 'rb') as f:
#     dict_ins_query_srl_triples = pickle.load(f)
# with open('dict_ins_doc_sent_srl_triples_v3.p', 'rb') as f:
#     dict_ins_doc_sent_srl_triples = pickle.load(f)

In [33]:
pretrained_weights = 'bert-large-cased-whole-word-masking'
train_dataset = Dataset(hotpot, list_hotpot_ner, dict_ins_doc_sent_srl_triples,
                            dict_ins_query_srl_triples, list_ent_query, 
                            dict_ins2dict_doc2pred=dict_ins2dict_doc2pred, batch_size=1,
                            pretrained_weights=pretrained_weights)

In [ ]:
(list_graphs,
        list_context,
        list_span_idx) = train_dataset.create_dataloader()

 82%|████████▏ | 6108/7405 [02:53<00:36, 35.07it/s]

In [ ]:
list_input_ids = [context['input_ids'] for context in list_context]
list_token_type_ids = [context['token_type_ids'] for context in list_context]
list_attention_masks = [context['attention_mask'] for context in list_context]

tensor_input_ids = torch.tensor(list_input_ids)
tensor_token_type_ids = torch.tensor(list_token_type_ids)
tensor_attention_masks = torch.tensor(list_attention_masks)

In [ ]:
model = HGNModel.from_pretrained(model_path)
model.cuda()

In [ ]:
validation = Validation(model, hotpot, list_graphs,
                        tensor_input_ids, tensor_attention_masks,
                        tensor_token_type_ids)
preds = validation.get_answer_predictions(dict_ins2dict_doc2pred)

In [ ]:
with open('pred_w_giwon_docs_last.json', 'w+') as f:
    json.dump(preds, f)